# Analyse BlueSeis BSPF Event Analysis

With pressure sensor parascientific and new sensor 

In [55]:
import os 
import obspy as obs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from andbro__savefig import __savefig

In [56]:
# def __request_data(seed, tbeg, tend):

#     from obspy.clients.fdsn import Client

#     client = Client("IRIS")

#     net, sta, loc, cha = seed.split(".")

#     try:
#         inventory = client.get_stations(network=net, 
#                                          station=sta,
#                                          starttime=tbeg-60,
#                                          endtime=tend+60,
#                                          level="response",
#                                          )
#     except:
#         print("Failed to load inventory!")
#         inventory = None
        
#     try:
#         waveform = client.get_waveforms(network=net,
#                                        station=sta,
#                                        location=loc,
#                                        channel=cha, 
#                                        starttime=tbeg-60,
#                                        endtime=tend+60,
#                                        )

#     except:
#         print("Failed to load waveforms!")
#         waveform = None
    
#     return waveform, inventory

In [57]:
def __get_theo_baz(config):
    
    from obspy.clients.fdsn import Client
    from obspy.geodetics.base import gps2dist_azimuth

    ## get event if not provided
    if 'event' not in config.keys():
        events = Client("USGS").get_events(starttime=config['eventtime']-20, endtime=config['eventtime']+20)
        if len(events) > 1:
            print(f" -> {len(events)} events found!!!")
            print(events)
            
    event = events[0]
    
#     print(event.event_descriptions[0]['type'], ': ',event.event_descriptions[0]['text'] + "\n")
    
    ## event location from event info
    config['source_latitude'] = event.origins[0].latitude
    config['source_longitude'] = event.origins[0].longitude
    
    
    dist, az, baz = gps2dist_azimuth(
                                    config['source_latitude'], config['source_longitude'], 
                                    config['BSPF_lat'], config['BSPF_lon'],
                                    )
    

    return baz

## Configurations

In [58]:
workdir = "/home/andbro/Documents/ROMY/BSPF"

In [59]:
config = {}

In [60]:
## blueSeis sensor
config['seed_blueseis'] = "PY.BSPF..HJ*"

## Trillium 240 next to BlueSeis on Pier
config['seed_seismometer'] = "II.PFO.10.BH*" 
# config['seed_seismometer'] = "PY.PFOIX..HH*" 

## BSPF coordinates
config['BSPF_lon'] = -116.455439
config['BSPF_lat'] = 33.610643


## Event Info

In [61]:
## Peru / Ecuador 
# config['tbeg'] = obs.UTCDateTime("2022-10-05 10:26")
# config['tend'] = obs.UTCDateTime("2022-10-05 10:40")

## Near PFO  - !! with XX location code
# config['tbeg'] = obs.UTCDateTime("2022-10-05 15:39:32")
# config['tend'] = obs.UTCDateTime("2022-10-05 15:39:45")

## ---- North Pacific Ocean -> No
# config['title'] = "North Pacific Ocean | 2022-10-25 18:40 UTC | M5.8"
# config['tbeg'] = obs.UTCDateTime("2022-11-02 04:53:00")
# config['tend'] = obs.UTCDateTime("2022-11-02 05:53:00")
# config['fmin'], config['fmax'] = 0.1, 0.5

## ---- Culf of California M5.1 -> No
# config['title'] = "Central California | 2022-10-25 18:40 UTC | M5.1"
# config['tbeg'] = obs.UTCDateTime("2022-10-25 18:40:00")
# config['tend'] = obs.UTCDateTime("2022-10-25 19:00:00")
# config['fmin'], config['fmax'] = 0.1, 0.5

## ---- Local M3.8-> No
# config['title'] = "Local | 2022-10-11 06:19 UTC | M3.8"
# config['tbeg'] = obs.UTCDateTime("2022-10-11 06:19:00")
# config['tend'] = obs.UTCDateTime("2022-10-11 06:30:00")
# config['fmin'], config['fmax'] = 0.5, 15.0


## Gulf of California M6.2 -> somewhat
# config['title'] = "Gullf of California | 2022-11-04 10:02 UTC | M6.2"
# config['tbeg'] = obs.UTCDateTime("2022-11-04 10:02")
# config['tend'] = obs.UTCDateTime("2022-11-04 10:15")
# config['fmin'], config['fmax'] = 0.025, 0.2

## ..., California M3.2 -> good
# config['tbeg'] = obs.UTCDateTime("2023-01-09 19:42:56")
# config['tend'] = config['tbeg']+20
# config['fmin'], config['fmax'] = 0.5, 10.0
# config['title'] = f"Borrego Springs, Ca | {config['tbeg'].date} {str(config['tbeg'].time)[:5]} UTC | M3.2"

## Las Brisas, Mexico M6.2 -> alright
# config['title'] = "Las Brisas, Mexico | 2022-11-22 16:39 UTC | M6.2"
# config['eventtime'] = obs.UTCDateTime("2022-11-22 16:39")
# config['tbeg'] = obs.UTCDateTime("2022-11-22 16:32")
# config['tend'] = obs.UTCDateTime("2022-11-22 17:00")
# config['fmin'], config['fmax'] = 0.01, 0.5

## Borrego Springs, California M4.2 -> good
config['eventtime'] = obs.UTCDateTime("2022-12-31 12:12:26")
config['tbeg'] = obs.UTCDateTime("2022-12-31 12:12:27")
config['tend'] = obs.UTCDateTime("2022-12-31 12:13")
config['fmin'], config['fmax'] = 0.1, 15.0
config['title'] = f"Borrego Springs, Ca | {config['tbeg'].date} {str(config['tbeg'].time)[:5]} UTC | M4.2"


        
## ... 
# config['tbeg'] = obs.UTCDateTime("2023-01-12 17:36:37")
# config['tend'] = config['tbeg']+15
# config['fmin'], config['fmax'] = 0.5, 10.0
# config['title'] = f"... | {config['tbeg'].date} {str(config['tbeg'].time)[:5]} UTC | M0.4"


## _________________________________________________

## ..., California M2.7
# config['tbeg'] = obs.UTCDateTime("2023-04-10 14:50:56")
# config['tend'] = config['tbeg']+25
# config['fmin'], config['fmax'] = 0.5, 80.0
# config['title'] = f"Borrego Springs, Ca | {config['tbeg'].date} {str(config['tbeg'].time)[:5]} UTC | M2.7"




## Load Data

In [62]:
from functions.request_data import __request_data

### Load blueSeis Data

In [64]:
py_bspf0, py_bspf_inv = __request_data(config['seed_blueseis'], config['tbeg'], config['tend'])


py_bspf = py_bspf0.copy()

## merge traces that might be split due to data gaps or overlaps
if len(py_bspf) > 3:
    py_bspf.merge(fill_value="interpolate")

## resample to 40 Hz
if config['seed_seismometer'].split(".")[3][0] == "B":
    py_bspf.resample(40)

## cut to event
py_bspf.trim(config['tbeg'], config['tend'])

## remove mean
py_bspf.detrend("demean")

## apply taper
py_bspf.taper(0.1)

## bandpass filter
py_bspf.filter("bandpass", freqmin=config['fmin'], freqmax=config['fmax'], corners=4, zerophase=True)

## show all traces in data stream
print(py_bspf)

py_bspf.plot(equal_scale=False);

Failed to load inventory!
Failed to load waveforms!


AttributeError: 'NoneType' object has no attribute 'copy'

### Load Seismometer Data

In [65]:
ii_pfo0, ii_pfo_inv = __request_data(config['seed_seismometer'], config['tbeg'], config['tend'])


ii_pfo = ii_pfo0.copy()

## cut properly
ii_pfo.trim(config['tbeg'], config['tend'])

## demean
ii_pfo.detrend("demean")

## taper 
ii_pfo.taper(0.1)

## bandpass
ii_pfo.filter("bandpass", freqmin=config['fmin'], freqmax=config['fmax'], corners=4, zerophase=True)

print(ii_pfo)
ii_pfo.plot(equal_scale=False);

Failed to load inventory!
Failed to load waveforms!


AttributeError: 'NoneType' object has no attribute 'copy'

### Join Data

In [ ]:
st_new = py_bspf.copy()
st_new += ii_pfo

### Plotting

In [ ]:
def __makeplot(config, st0):

    st = st0.copy()
    
    fig, ax = plt.subplots(len(st),1, figsize=(15,10), sharex=True)

    font = 14

    time_scaling, time_unit = 1, "sec"
    rot_scaling = 1e9
    trans_scaling = 1e6
    
    for i, tr in enumerate(st):

        if i in [0,1,2]:
            ax[i].set_ylabel(r"$\omega$ (nrad/s)", fontsize=font)
            ax[i].plot(tr.times()/time_scaling, tr.data*rot_scaling, 'k', label=tr.stats.station+"."+tr.stats.channel)

        elif i in [3,4,5]:
            ax[i].set_ylabel(r"u ($\mu$m/s)", fontsize=font)
            ax[i].plot(tr.times()/time_scaling, tr.data*trans_scaling, 'k', label=tr.stats.station+"."+tr.stats.channel)
        
        elif i in [6,7,8]:
            ax[i].set_ylabel(r"$\omega$ (nrad/s)", fontsize=font)
            ax[i].plot(tr.times()/time_scaling, tr.data*rot_scaling, 'k', label=tr.stats.station+"."+tr.stats.channel)
            
        ax[i].legend(loc=1)
        
    ax[len(st)-1].set_xlabel(f"Time ({time_unit}) from {st[0].stats.starttime.date} {str(st[0].stats.starttime.time).split('.')[0]} UTC", fontsize=font)
    ax[0].set_title(config['title']+f" | {config['fmin']} - {config['fmax']} Hz", fontsize=font, pad=10)
    
    plt.show();
    return fig

In [ ]:
fig = __makeplot(config, st_new)

In [ ]:
# __savefig(fig, outpath="/home/andbro/Downloads/tmp/BSPF/", outname="BSPF_event_1", mode="png", dpi=300)

In [ ]:
def __makeplotStreamSpectra2(st, config, fscale=None):

    from scipy import fftpack
    from andbro__fft import __fft
    import matplotlib.pyplot as plt

    NN = len(st)
    rot_scaling, rot_unit = 1e6, r"$\mu$rad/s"
    trans_scaling, trans_unit = 1e3, r"mm/s"
        
    fig, axes = plt.subplots(NN,2,figsize=(15,int(NN*2)), sharex='col')

    font = 14
    
    plt.subplots_adjust(hspace=0.3)

    ## _______________________________________________

    st.sort(keys=['channel'], reverse=True)
    
    for i, tr in enumerate(st):

#         comp_fft = abs(fftpack.fft(tr.data))
#         ff       = fftpack.fftfreq(comp_fft.size, d=1/tr.stats.sampling_rate)
#         comp_fft = fftpack.fftshift(comp_fft)
#         ff, spec = ff[1:len(ff)//2], abs(fftpack.fft(tr.data)[1:len(ff)//2])

        if tr.stats.channel[-2] == "J":
            scaling = rot_scaling
        elif tr.stats.channel[-2] == "H":
            scaling = trans_scaling

        spec, ff, ph = __fft(tr.data*scaling, tr.stats.delta, window=None, normalize=None)
    
    
        ## _________________________________________________________________
        if tr.stats.channel[-2] == "J":
            axes[i,0].plot(
                        tr.times(),
                        tr.data*rot_scaling,
                        color='black',
                        label='{} {}'.format(tr.stats.station, tr.stats.channel),
                        lw=1.0,
                        )
            axes[i,1].fill_between(ff, 0, spec, alpha=0.9, color='black', lw=0)


        elif tr.stats.channel[-2] == "H":
            axes[i,0].plot(
                        tr.times(),
                        tr.data*trans_scaling,
                        color='darkblue',
                        label='{} {}'.format(tr.stats.station, tr.stats.channel),
                        lw=1.0,
                        )
            axes[i,1].fill_between(ff, 0, spec, alpha=0.9, color='darkblue', lw=0)


        ## _________________________________________________________________
#         if fscale == "loglog":
# #             axes[i,1].loglog(ff, spec, color='black', lw=1.0)
#             axes[i,1].fill_between(ff, 0, spec, alpha=0.7, color='black', lw=1.)
            
#         elif fscale == "loglin":
# #             axes[i,1].semilogx(ff, spec, color='black', lw=1.0)
#             axes[i,1].fill_between(ff, 0, spec, alpha=0.7, color='black', lw=1.)

#         elif fscale == "linlog":
# #             axes[i,1].semilogy(ff, spec, color='black', lw=1.0)
#             axes[i,1].fill_between(ff, 0, spec, alpha=0.7, color='black', lw=1.)

#         else:
# #             axes[i,1].plot(ff, spec, color='black', lw=1.0)
#             axes[i,1].fill_between(ff, 0, spec, alpha=0.7, color='black')

        
        if tr.stats.channel[1] == "J":
            sym, unit = r"$\Omega$", rot_unit
        elif tr.stats.channel[1] == "H":
            sym, unit = "v", trans_unit
        else:
            unit = "Amplitude", "a.u."
            
        axes[i,0].set_ylabel(f'{sym} ({unit})',fontsize=font)    
        axes[i,1].set_ylabel(f'ASD \n({unit}/Hz)',fontsize=font)        
        axes[i,0].legend(loc='upper left',bbox_to_anchor=(0.8, 1.10), framealpha=1.0)
        
#         axes[i,0].ticklabel_format(axis='y', style='sci', scilimits=(0,0))
#         axes[i,1].ticklabel_format(axis='y', style='sci', scilimits=(0,0))

    if "fmin" in config.keys() and "fmax" in config.keys():
        axes[i,1].set_xlim(config['fmin'],config['fmax'])

    axes[NN-1,0].set_xlabel(f"Time from {tr.stats.starttime.date} {str(tr.stats.starttime.time)[:8]} (s)",fontsize=font)     
    axes[NN-1,1].set_xlabel(f"Frequency (Hz)",fontsize=font)     

    return fig

fig = __makeplotStreamSpectra2(st_new, config, fscale="linlin");

In [ ]:
# __savefig(fig, outpath="/home/andbro/Downloads/tmp/BSPF/", outname="BSPF_ASD_event_1", mode="png", dpi=300)

In [ ]:
def __makeplotStreamSpectra3(st0, config):

    from scipy import fftpack
    from andbro__fft import __fft
    from matplotlib.gridspec import GridSpec
    import matplotlib.pyplot as plt

    st = st0.copy()
    
    NN = 5
    rot_scaling, rot_unit = 1e9, r"nrad/s"
    trans_scaling, trans_unit = 1e6, r"$\mu$m/s"
    
    rot_scaling, rot_unit = 1, r"nrad/s"
    trans_scaling, trans_unit = 1, r"$\mu$m/s"

    colors = ['crimson', 'red', 'magenta', 'green', 'yellowgreen', 'lime']

#     fig, axes = plt.subplots(NN, 2 ,figsize=(15, int(NN*2)), sharex='col', layout="constrained")
    fig = plt.figure(figsize=(15,12))
    
    gs = GridSpec(NN, 2, figure=fig)
    
    ax1 = fig.add_subplot(gs[0, :-1])
    ax2 = fig.add_subplot(gs[1, :-1])
    ax3 = fig.add_subplot(gs[2, :-1])

    ax4 = fig.add_subplot(gs[0, 1:])
    ax5 = fig.add_subplot(gs[1, 1:])
    ax6 = fig.add_subplot(gs[2, 1:])

    ax7 = fig.add_subplot(gs[3:, :])
    
    
    font = 14
    
    plt.subplots_adjust(hspace=0.1)

    ## _______________________________________________

    st.sort(keys=['channel'], reverse=True)
    
    ffs, specs, names = [], [], []
    for i, tr in enumerate(st):

        tr = tr.normalize()
        
        if tr.stats.channel[-2] == "J":
            scaling = rot_scaling
        elif tr.stats.channel[-2] == "H":
            scaling = trans_scaling

        spec, ff, ph = __fft(tr.data*scaling, tr.stats.delta, window=None, normalize=True)
        
        ffs.append(ff)
        specs.append(spec)
        names.append(f"{tr.stats.station}.{tr.stats.channel}")
    
    hz = st.select(channel="*HZ")[0]
    hn = st.select(channel="*HN")[0]
    he = st.select(channel="*HE")[0]

    jz = st.select(channel="*JZ")[0]
    jn = st.select(channel="*JN")[0]
    je = st.select(channel="*JE")[0]

    ax1.plot(hz.times(), hz.data*trans_scaling, 'k', label=f"{hz.stats.station}.{hz.stats.channel}")
    ax2.plot(hn.times(), hn.data*trans_scaling, 'k', label=f"{hn.stats.station}.{hn.stats.channel}")
    ax3.plot(he.times(), he.data*trans_scaling, 'k', label=f"{he.stats.station}.{he.stats.channel}")

    ax4.plot(jz.times(), jz.data*rot_scaling, 'k', label=f"{jz.stats.station}.{jz.stats.channel}")
    ax5.plot(jn.times(), jn.data*rot_scaling, 'k', label=f"{jn.stats.station}.{jn.stats.channel}")
    ax6.plot(je.times(), je.data*rot_scaling, 'k', label=f"{je.stats.station}.{je.stats.channel}")

    
    for ii, (ff, spec, name) in enumerate(zip(ffs, specs, names)):
#         ax7.plot(ff, spec, label=name, alpha=0.6, color=colors[ii])
        ax7.fill_between(ff, 0, spec, label=name, alpha=0.2, color=colors[ii])
    
    
    for ax in [ax1, ax2, ax3, ax4 ,ax5, ax6]:
        ax.legend(loc=1, fontsize=font-2)
        ax.set_ylim(-1,1)
        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)
        ax.spines['bottom'].set_visible(False)
        if ax not in [ax3, ax6]:
            ax.set(xticklabels=[]) 
            ax.tick_params(bottom=False)  
        
        
    if "fmin" in config.keys() and "fmax" in config.keys():
        ax7.set_xlim(config['fmin'], config['fmax'])    
    
    ax7.legend(loc=2, fontsize=font-2)

    ax1.set_ylabel("norm. acc. (m/s)", fontsize=font-1)
    ax2.set_ylabel("norm. acc. (m/s)", fontsize=font-1)
    ax3.set_ylabel("norm. acc. (m/s)", fontsize=font-1)

    ax4.set_ylabel("norm. rot. (rad/s)", fontsize=font-1)
    ax5.set_ylabel("norm. rot. (rad/s)", fontsize=font-1)
    ax6.set_ylabel("norm. rot. (rad/s)", fontsize=font-1)

    ax3.set_xlabel("Time (s)", fontsize=font)
    ax6.set_xlabel("Time (s)", fontsize=font)

    ax7.set_xlabel("Frequency (Hz)", fontsize=font)
    ax7.set_ylabel("Spec. Amplitude ()", fontsize=font)
    
    
    bbox = list(ax7.get_position().bounds)
    bbox[3] = .9*bbox[3] # Reduce the height of the axis a bit.
    ax7.set_position(bbox)
    
#     axes[i,0].ticklabel_format(axis='y', style='sci', scilimits=(0,0))
#     axes[i,1].ticklabel_format(axis='y', style='sci', scilimits=(0,0))

    return fig

In [ ]:
fig = __makeplotStreamSpectra3(st_new, config);

### ZRT

In [ ]:
st_new

In [ ]:
baz = __get_theo_baz(config)

In [ ]:
from obspy.signal.rotate import rotate_ne_rt

## rotate acc NE to RT
if "PFOIX" in config['seed_seismometer']:
    acc = st_new.copy().select(station="PFOIX")
else:
    acc = st_new.copy().select(station="PFO")

    
acc_N = acc.select(channel='*N')[0]
acc_E = acc.select(channel='*E')[0]

acc_N.data, acc_E.data = rotate_ne_rt(acc_N.data, acc_E.data, baz)

# acc_N.stats.channel = "BHR"
# acc_E.stats.channel = "BHT"

## rotate rot NE to RT   
rot = st_new.copy().select(station="BSPF")

rot_N = rot.select(channel='*N')[0]
rot_E = rot.select(channel='*E')[0]

rot_N.data, rot_E.data = rotate_ne_rt(rot_N.data, rot_E.data, baz)

# rot_N.stats.channel = "HJR"
# rot_E.stats.channel = "HJT"

st_zrt = obs.Stream()

st_zrt += st_new.copy().select(channel="*HZ")[0]
st_zrt += st_new.copy().select(channel="*JZ")[0]
st_zrt += acc_N
st_zrt += acc_E
st_zrt += rot_N
st_zrt += rot_E

# st_zrt.plot();

In [ ]:
fig = __makeplotStreamSpectra3(st_zrt, config);

## Load ADR 

In [ ]:
from functions.compute_adr_pfo import __compute_adr_pfo

In [ ]:
adr = __compute_adr_pfo(config['tbeg'], config['tend'])

In [ ]:
## taper 
adr.taper(0.01)

## bandpass
adr.filter("bandpass", freqmin=config['fmin'], freqmax=config['fmax'], corners=4, zerophase=True)

In [ ]:
def __makeplot2(config, st0):

    st = st0.copy()
    
    fig, ax = plt.subplots(len(st),1, figsize=(15,10), sharex=True)

    font = 14

    time_scaling, time_unit = 1, "sec"
    rot_scaling = 1
    trans_scaling = 1
    
    for i, tr in enumerate(st):

        if i in [0,1,2]:
#             ax[i].set_ylabel(r"$\omega$ (nrad/s)", fontsize=font)
            ax[i].plot(tr.times()/time_scaling, tr.data*rot_scaling, 'black', label=tr.stats.station+"."+tr.stats.channel)

        elif i in [3,4,5]:
#             ax[i].set_ylabel(r"u ($\mu$m/s)", fontsize=font)
            ax[i].plot(tr.times()/time_scaling, tr.data*trans_scaling, 'darkred', label=tr.stats.station+"."+tr.stats.channel)
        
        elif i in [6,7,8]:
#             ax[i].set_ylabel(r"$\omega$ (nrad/s)", fontsize=font)
            ax[i].plot(tr.times()/time_scaling, tr.data*rot_scaling, 'darkgreen', label=tr.stats.station+"."+tr.stats.channel)
         
        ax[i].set_ylim(-1,1)
        ax[i].spines['right'].set_visible(False)
        ax[i].spines['top'].set_visible(False)
        if i != int(len(st)-1):
            ax[i].spines['bottom'].set_visible(False)
#             ax[i].set(xticklabels=[]) 
            ax[i].tick_params(bottom=False)
        
        ax[i].legend(loc=1)
        
    ax[len(st)-1].set_xlabel(f"Time ({time_unit}) from {st[0].stats.starttime.date} {str(st[0].stats.starttime.time).split('.')[0]} UTC", fontsize=font)
    ax[0].set_title(config['title']+f" | {config['fmin']} - {config['fmax']} Hz", fontsize=font, pad=10)
    
    plt.show();
    return fig

In [ ]:
st_all = st_new.copy()
st_all += adr.copy()

st_all = st_all.trim(adr[0].stats.starttime, adr[0].stats.endtime)

st_all = st_all.normalize()

fig = __makeplot2(config, st_all)

In [ ]:
st_adr = obs.Stream()
st_adr += py_bspf.copy()
st_adr += adr.copy()

st_adr = st_adr.trim(adr[0].stats.starttime, adr[0].stats.endtime)

### Plot Coherence

In [ ]:
def __makeplotStreamSpectra4(st0, config):

    from scipy import fftpack
    from andbro__fft import __fft
    from matplotlib.gridspec import GridSpec
    from andbro__compute_coherence_stream import __coherence_stream
    from numpy import array
    
    import matplotlib.pyplot as plt

    st = st0.copy()
    
    NN = 5
    rot_scaling, rot_unit = 1e9, r"nrad/s"
    trans_scaling, trans_unit = 1e6, r"$\mu$m/s"
    
    rot_scaling, rot_unit = 1, r"nrad/s"
    trans_scaling, trans_unit = 1, r"$\mu$m/s"

    colors = ['crimson', 'red', 'magenta', 'darkblue', 'navy', 'indigo']

    fig = plt.figure(figsize=(15,12))
    
    gs = GridSpec(NN, 2, figure=fig)
    
    ax1 = fig.add_subplot(gs[0, :-1])
    ax2 = fig.add_subplot(gs[1, :-1])
    ax3 = fig.add_subplot(gs[2, :-1])

    ax4 = fig.add_subplot(gs[0, 1:])
    ax5 = fig.add_subplot(gs[1, 1:])
    ax6 = fig.add_subplot(gs[2, 1:])

    ax7 = fig.add_subplot(gs[3:, :])
    
    
    font = 14
    
    plt.subplots_adjust(hspace=0.1)

    ## _______________________________________________

    st.sort(keys=['channel'], reverse=True)
    
    ff, spec = __coherence_stream(st, st.select(channel="HJZ"), nseg=512, nover=50)
    
    names, ffs, specs = [], [], []
    for i, tr in enumerate(st):
        tr = tr.normalize()
        
        ffs.append(ff[i])
        specs.append(spec[i][0])
        names.append(f"{tr.stats.station}.{tr.stats.channel}")
        
        
    hz = st.select(channel="BJZ")[0]
    hn = st.select(channel="BJN")[0]
    he = st.select(channel="BJE")[0]

    jz = st.select(channel="HJZ")[0]
    jn = st.select(channel="HJN")[0]
    je = st.select(channel="HJE")[0]

    ax1.plot(hz.times(), hz.data*trans_scaling, 'k', label=f"{hz.stats.station}.{hz.stats.channel}")
    ax2.plot(hn.times(), hn.data*trans_scaling, 'k', label=f"{hn.stats.station}.{hn.stats.channel}")
    ax3.plot(he.times(), he.data*trans_scaling, 'k', label=f"{he.stats.station}.{he.stats.channel}")

    ax4.plot(jz.times(), jz.data*rot_scaling, 'k', label=f"{jz.stats.station}.{jz.stats.channel}")
    ax5.plot(jn.times(), jn.data*rot_scaling, 'k', label=f"{jn.stats.station}.{jn.stats.channel}")
    ax6.plot(je.times(), je.data*rot_scaling, 'k', label=f"{je.stats.station}.{je.stats.channel}")

    
    for ii, (ff, spec, name) in enumerate(zip(ffs, specs, names)):
        ax7.plot(ff, spec, label=name, alpha=0.6, color=colors[ii])
    
    
    for ax in [ax1, ax2, ax3, ax4 ,ax5, ax6]:
        ax.legend(loc=1, fontsize=font-2)
        ax.set_ylim(-1,1)
        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)
        ax.spines['bottom'].set_visible(False)
        if ax not in [ax3, ax6]:
            ax.set(xticklabels=[]) 
            ax.tick_params(bottom=False)  
        
        
    if "fmin" in config.keys() and "fmax" in config.keys():
        ax7.set_xlim(config['fmin'], config['fmax'])    
    
    ax7.legend(loc=2, fontsize=font-2)

    ax3.set_xlabel("Time (s)", fontsize=font)
    ax6.set_xlabel("Time (s)", fontsize=font)

    ax7.set_xlabel("Frequency (Hz)", fontsize=font)
    ax7.set_ylabel("Spec. Amplitude ()", fontsize=font)
    
    bbox = list(ax7.get_position().bounds)
    bbox[3] = .9*bbox[3] # Reduce the height of the axis a bit.
    ax7.set_position(bbox)
    
#     axes[i,0].ticklabel_format(axis='y', style='sci', scilimits=(0,0))
#     axes[i,1].ticklabel_format(axis='y', style='sci', scilimits=(0,0))

    return fig

In [ ]:
fig = __makeplotStreamSpectra2(st_adr, config);

In [ ]:
fig = __makeplotStreamSpectra4(st_adr, config);

### Plot CrossCorrelation

In [ ]:
py_z = st_adr[0]
ii_z = st_adr[3]

In [ ]:
py_z.plot(); ii_z.plot();

In [ ]:
from functions.compute_cc_for_fbands import __compute_cc_for_fbands

In [ ]:
# def __compute_cc_for_fbands(tr1, tr2, fmin=1, fmax=20, fband_type='octave'):
    
#     def __get_frequency_bands(f_min=fmin, f_max=fmax, fband_type=fband_type):

#         from numpy import sqrt, array

#         f_lower, f_upper, f_centers = [], [], []
#         fcenter = f_max

#         while fcenter > f_min:
#             f_lower.append(fcenter/(sqrt(sqrt(2.))))
#             f_upper.append(fcenter*(sqrt(sqrt(2.))))
#             f_centers.append(fcenter)

#             fcenter = fcenter/(sqrt(2.))

#         return array(f_lower), array(f_upper), array(f_centers)
    
    
#     from numpy import array, corrcoef

#     f_lower, f_upper, f_centers = __get_frequency_bands(f_min=fmin, f_max=fmax, fband_type=fband_type)


#     ccorrs = []
#     for fl, fu in zip(f_lower, f_upper):

#         tr01 = tr1.copy();
#         tr02 = tr2.copy();

#         tr01.detrend("linear")
#         tr02.detrend("linear")

#         tr01.normalize();
#         tr02.normalize();
        
#         tr01.taper(0.1);
#         tr02.taper(0.1);
        
#         tr01 = tr01.filter("bandpass", freqmin=fl, freqmax=fu, corners=8, zerophase=True);
#         tr02 = tr02.filter("bandpass", freqmin=fl, freqmax=fu, corners=8, zerophase=True);

#         tr01.normalize();
#         tr02.normalize();        
        
#         ccorrs.append(corrcoef(tr01.data, tr02.data)[0][1])

# #         plt.figure()
# #         plt.plot(tr01.data)
# #         plt.plot(tr02.data) 
    
#     return array(f_centers), array(ccorrs)

In [ ]:
f_centers, ccorrs = __compute_cc_for_fbands(py_z, ii_z, fmin=config['fmin'], fmax=config['fmax'], fband_type='octave')

plt.figure()
plt.scatter(f_centers, ccorrs)
plt.plot(f_centers, ccorrs)
plt.xscale("log")

In [ ]:
def __makeplotStreamSpectra5(st0, config):

    from scipy import fftpack
    from andbro__fft import __fft
    from matplotlib.gridspec import GridSpec
    from andbro__crosscorrelation import __crosscorrelation
    from numpy import array
    
    import matplotlib.pyplot as plt

    st = st0.copy()
    
    NN = 5
    rot_scaling, rot_unit = 1e9, r"nrad/s"
    trans_scaling, trans_unit = 1e6, r"$\mu$m/s"
    
    rot_scaling, rot_unit = 1, r"nrad/s"
    trans_scaling, trans_unit = 1, r"$\mu$m/s"

    colors = ['crimson', 'red', 'magenta', 'darkblue', '#3776ab', 'indigo']

    fig = plt.figure(figsize=(15,12))
    
    gs = GridSpec(NN, 2, figure=fig)
    
    ax1 = fig.add_subplot(gs[0, :-1])
    ax2 = fig.add_subplot(gs[1, :-1])
    ax3 = fig.add_subplot(gs[2, :-1])

    ax4 = fig.add_subplot(gs[0, 1:])
    ax5 = fig.add_subplot(gs[1, 1:])
    ax6 = fig.add_subplot(gs[2, 1:])

    ax7 = fig.add_subplot(gs[3:, :])
    
    
    font = 14
    
    plt.subplots_adjust(hspace=0.1)

    ## _______________________________________________

    st.normalize()
    
    st.sort(keys=['channel'], reverse=True)
        
    names, ffs, ccs = [], [], []
    for comp in ["Z", "N", "E"]:
        
        tmp = st.select(channel=f"*{comp}")

        f_centers, ccorrs = __compute_cc_for_fbands(tmp[0], tmp[1], 
                                                    fmin=config['fmin'], 
                                                    fmax=config['fmax'], 
                                                    fband_type='octave'
                                                   )

        names.append(f"{tmp[0].stats.channel}")
        ffs.append(f_centers)
        ccs.append(ccorrs)
        
    hz = st.select(channel="BJZ")[0]
    hn = st.select(channel="BJN")[0]
    he = st.select(channel="BJE")[0]

    jz = st.select(channel="HJZ")[0]
    jn = st.select(channel="HJN")[0]
    je = st.select(channel="HJE")[0]

    ax1.plot(hz.times(), hz.data*trans_scaling, colors[3], label=f"{hz.stats.station}.{hz.stats.channel}")
    ax2.plot(hn.times(), hn.data*trans_scaling, colors[4], label=f"{hn.stats.station}.{hn.stats.channel}")
    ax3.plot(he.times(), he.data*trans_scaling, colors[5], label=f"{he.stats.station}.{he.stats.channel}")

    ax4.plot(jz.times(), jz.data*rot_scaling, colors[3], label=f"{jz.stats.station}.{jz.stats.channel}")
    ax5.plot(jn.times(), jn.data*rot_scaling, colors[4], label=f"{jn.stats.station}.{jn.stats.channel}")
    ax6.plot(je.times(), je.data*rot_scaling, colors[5], label=f"{je.stats.station}.{je.stats.channel}")

    
    for ii, (ff, cc, name) in enumerate(zip(ffs, ccs, names)):
#         ax7.plot(ff, spec, label=name, alpha=0.6, color=colors[ii])
        ax7.plot(ff, cc, label=name[-1], alpha=0.6, color=colors[ii+3])
        ax7.scatter(ff, cc, alpha=0.6, color=colors[ii+3])
        ax7.set_xscale("log")
    
    for ax in [ax1, ax2, ax3, ax4 ,ax5, ax6]:
        ax.legend(loc=1, fontsize=font-2)
        ax.set_ylim(-1,1)
        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)
        ax.spines['bottom'].set_visible(False)
        if ax not in [ax3, ax6]:
            ax.set(xticklabels=[]) 
            ax.tick_params(bottom=False)  
        
        
#     if "fmin" in config.keys() and "fmax" in config.keys():
#         ax7.set_xlim(config['fmin'], config['fmax'])    
    
    ax7.legend(loc=4, fontsize=font-2)

    ax3.set_xlabel("Time (s)", fontsize=font)
    ax6.set_xlabel("Time (s)", fontsize=font)

    ax7.set_xlabel("Frequency (Hz)", fontsize=font)
    ax7.set_ylabel("Spec. Amplitude ()", fontsize=font)
    
    bbox = list(ax7.get_position().bounds)
    bbox[3] = .9*bbox[3] # Reduce the height of the axis a bit.
    ax7.set_position(bbox)
    
#     axes[i,0].ticklabel_format(axis='y', style='sci', scilimits=(0,0))
#     axes[i,1].ticklabel_format(axis='y', style='sci', scilimits=(0,0))

    return fig

In [ ]:
fig = __makeplotStreamSpectra5(st_adr, config);

## Backazimuth

In [ ]:
config['station_longitude'] = config['BSPF_lon']
config['station_latitude']  = config['BSPF_lat']

config['win_length_sec'] = .5  ## window length for correlation
config['step'] = 1
config['overlap'] = 25

In [ ]:
from functions.compute_backazimuth import __compute_backazimuth

out = __compute_backazimuth(ii_pfo, py_bspf, config, plot=True)

In [ ]:
# def __compute_backazimuth(st_acc, st_rot, config, event=None, plot=True):
    
#     import numpy as np
#     import matplotlib as mpl
    
#     from obspy import read, read_events, UTCDateTime
#     from obspy.clients.fdsn import Client
#     from obspy.geodetics.base import gps2dist_azimuth
#     from obspy.signal.cross_correlation import xcorr
#     from obspy.signal.rotate import rotate_ne_rt

#     ## _______________________________    
#     ## check config
#     keywords=['tbeg','tend','station_latitude', 'station_longitude',
#               'step', 'win_length_sec', 'overlap', 'eventtime']

#     for key in keywords: 
#         if key not in config.keys():
#             print(f" -> {key} is missing in config!\n")
            
            
#     ## _______________________________    
#     ## Defaults
#     if 'win_length_sec' not in config.keys():
#         config['win_length_sec'] = .5    ## window length for correlation
#     if 'step' not in config.keys():
#         config['step'] = 1
#     if 'overlap' not in config.keys():
#         config['overlap'] = 25
    
    
#     ## time period
#     config['tbeg'], config['tend'] = UTCDateTime(config['tbeg']), UTCDateTime(config['tend'])

#     ## _______________________________    
#     ## streams
#     ACC = st_acc.trim(config['tbeg'], config['tend'])
#     ROT = st_rot.trim(config['tbeg'], config['tend']).select(channel="*Z")
    
#     ## _______________________________    
#     ## get event if not provided
#     if not event:
#         events = Client("USGS").get_events(starttime=config['eventtime']-20, endtime=config['eventtime']+20)
#         if len(events) > 1:
#             print(f" -> {len(events)} events found!!!")
#             print(events)
            
#     event = events[0]
    
#     ## event location from event info
#     config['source_latitude'] = event.origins[0].latitude
#     config['source_longitude'] = event.origins[0].longitude
    
#     print(event.event_descriptions[0]['type'], ': ',event.event_descriptions[0]['text'] + "\n")
    

#     ## _______________________________        
#     ## theoretical backazimuth and distance
    
#     config['baz'] = gps2dist_azimuth(
#                                     config['source_latitude'], config['source_longitude'], 
#                                     config['station_latitude'], config['station_longitude'],
#                                     )

#     print('Epicentral distance [m]:       ',np.round(config['baz'][0],1))
#     print('Theoretical azimuth [deg]:     ', np.round(config['baz'][1],1))
#     print('Theoretical backazimuth [deg]: ', np.round(config['baz'][2],1))

#     ## _______________________________    
#     ## backazimuth estimation
    
#     config['sampling_rate'] = int(ROT.select(channel="*Z")[0].stats.sampling_rate)
    
#     config['num_windows'] = len(ROT.select(channel="*Z")[0]) // (int(config['sampling_rate'] * config['win_length_sec']))

    
#     backas = np.linspace(0, 360 - config['step'], int(360 / config['step']))
   
#     corrbaz = []
    
#     ind = None
    
#     for i_deg in range(0, len(backas)):
        
#         for i_win in range(0, config['num_windows']):
            
#             ## infer indices
#             idx1 = int(config['sampling_rate'] * config['win_length_sec'] * i_win)
#             idx2 = int(config['sampling_rate'] * config['win_length_sec'] * (i_win + 1))
            
#             ## add overlap
#             if i_win > 0 and i_win < config['num_windows']:
#                 idx1 = int(idx1 - config['overlap']/100 * config['win_length_sec'] * config['sampling_rate'])
#                 idx2 = int(idx2 + config['overlap']/100 * config['win_length_sec'] * config['sampling_rate'])
                    
#             ## rotate NE to RT   
#             R, T = rotate_ne_rt(ACC.select(channel='*N')[0].data, 
#                                 ACC.select(channel='*E')[0].data,
#                                 backas[i_deg]
#                                )
            
#             ## compute correlation for backazimuth
#             corrbaz0 = xcorr(ROT.select(channel="*Z")[0][idx1:idx2], 
#                              T[idx1:idx2], 0,
#                               )
            
#             corrbaz.append(corrbaz0[1])

            
#     corrbaz = np.asarray(corrbaz)
#     corrbaz = corrbaz.reshape(len(backas), config['num_windows'])


#     ## extract maxima
#     maxcorr = np.array([backas[corrbaz[:, l1].argmax()] for l1 in range(0, config['num_windows'])])

#     ## create mesh grid
#     mesh = np.meshgrid(np.arange(config['win_length_sec']/2, config['win_length_sec'] * config['num_windows'], config['win_length_sec']), backas)

    

#     ## _______________________________
#     ## Plotting
#     def __makeplot():
    
#         ## get rotated acceleration
#         R, T = rotate_ne_rt(ACC.select(component='N')[0].data, 
#                             ACC.select(component='E')[0].data,
#                             config['baz'][2]
#                            )

#         fig, ax = plt.subplots(3, 1, figsize=(15, 10))

#         ## parameters
#         font = 14
#         acc_scaling, acc_unit = 1e3, "mm/s$^2$"
#         rot_scaling, rot_unit = 1e6, "$\mu$rad/s"

#         ## create time axis
#         time = np.linspace(0, len(ACC[0].data)/ACC[0].stats.sampling_rate, len(ACC[0].data))

#         ## plot vertical rotation rate
#         ax[0].plot(time, ROT.select(channel="*Z")[0].data*rot_scaling, label='vertical rotation rate')

#         ax[0].set_xlim(time[0], time[-1])
#         ax[0].set_ylabel(f'vert. rot. rate \n({rot_unit})', fontsize=font)
#         ax[0].legend()

#         # add P- and S-wave arrivals
#         ROT_max = max(ROT.select(channel="*Z")[0].data*rot_scaling)
        
#         ## plot transverse acceleration
#         ax[1].plot(time, T*acc_scaling, 'k',label='transverse acceleration')
#         ax[1].set_xlim(time[0], time[-1])
#         ax[1].set_ylabel(f'transv. acc. \n({acc_unit})', fontsize=font)
#         ax[1].ticklabel_format(axis='y', style='sci', scilimits=(-2,2))
#         ax[1].legend()
        

#         ## backazimuth estimation plot
#         im = ax[2].pcolormesh(mesh[0], mesh[1], corrbaz, cmap=plt.cm.RdYlGn_r, vmin=-1, vmax=1, shading="auto")
#         ax[2].set_xlim(time[0], time[-1])
#         ax[2].set_ylim(0, 360)
#         ax[2].set_ylabel(u'estimated \nbackazimuth (°)', fontsize=font)
#         ax[2].set_xlabel('time (s)', fontsize=font)

#         ## plot maximal correclation values
#         ax[2].plot(np.arange(config['win_length_sec']/2., config['win_length_sec'] * len(maxcorr), config['win_length_sec']), maxcorr, '.k')

#         ## plot theoretical Backazimuth for comparison
#         xx = np.arange(0, config['win_length_sec'] * len(maxcorr) + 1, config['win_length_sec'])
#         tba = np.ones(len(xx)) * config['baz'][2]
#         if config['baz'][2] < 330:
#             x_text, y_text = time[int(0.82*len(time))], config['baz'][2]+5
#         else:
#             x_text, y_text = time[int(0.82*len(time))], config['baz'][2]-15

#         ax[2].plot(xx, tba, c='.5', lw=1.5, alpha=0.6)
#         ax[2].text(x_text, y_text, u'Theor. BAz = '+str(round(config['baz'][2],2))+'°', color='k', fontsize=font-1)

#         edist = round(config['baz'][0]/1000,1)
        
#         if 'fmin' in config.keys() and 'fmax' in config.keys():
#             ax[0].set_title(config['title'] +f" | {edist} km" + f" | {config['fmin']}-{config['fmax']} Hz", pad=15, fontsize=font)
#         else:
#             ax[0].set_title(config['title'] +f" | {edist} km", pad=15, fontsize=font)
 
            
#         ## add colorbar
#     #     norm = mpl.colors.Normalize(vmin=-1, vmax=1)
#     #     cb1 = mpl.colorbar.ColorbarBase(fig, cmap=plt.cm.RdYlGn_r, norm=norm, orientation='vertical')
#         cax = ax[2].inset_axes([1.01, 0., 0.02, 1])
#         cb1 = plt.colorbar(im, ax=ax[2], cax=cax)

#         plt.show();
#         return fig
    
    
#     if plot:
#         __makeplot();
        

#     ## _______________________________
#     ## prepare output        
    
#     output = {}
    
#     output['baz_mesh'] = mesh
#     output['baz_corr'] = corrbaz
#     output['baz_theo'] = config['baz']
#     output['acc_transverse'] = T
#     output['acc_radial'] = R
#     output['rot_vertical'] = ROT.select(channel="*Z")
#     output['event'] = event
    
#     return output
        
# out = __compute_backazimuth(ii_pfo, py_bspf, config, plot=True)

In [69]:
from obspy.clients.fdsn import Client

seed = "PY.BSPF..HJ*"
tbeg = obs.UTCDateTime("2022-10-01 00:00")
tend = obs.UTCDateTime("2022-10-01 00:00") + 60


client = Client("IRIS")

net, sta, loc, cha = seed.split(".")


# inventory = client.get_stations(network=net, 
#                                  station=sta,
#                                  starttime=tbeg-60,
#                                  endtime=tend+60,
#                                  level="response",
#                                  )
waveform = client.get_waveforms(network=net,
                               station=sta,
                               location=loc,
                               channel=cha, 
                               starttime=tbeg-60,
                               endtime=tend+60,
                               )


FDSNException: Service responds: Internal server error
Detailed response of server:

Error 500: Error calling internal metadata service, code 404 [<!doctype html>
<html lang="en">
  <head>
    
      <title>
        
          IRIS: 
  404 Page Not Found Error
        
      </title>
      
  <meta name="keywords" content="seismology, seismological data,
  earthquakes, iris dmc" />
  <!-- Forces IE to render in 'latest' compatible mode -->
  <meta http-equiv="X-UA-Compatible" content="IE=Edge"/>
  <meta charset="utf-8" />
  <meta name="viewport" content="width=device-width, initial-scale=1" />
  
      
  <link rel="stylesheet" media="all" type="text/css" href="/static/css/iris.css" />
  <link rel="stylesheet" media="print" type="text/css" href="/static/css/print.css" />
  
  
  
    <link rel="shortcut icon" href="/static/img/favicon.ico" />
  
      
  <script type="text/javascript" src="//ajax.googleapis.com/ajax/libs/jquery/1.11.1/jquery.js"></script>
  <script type="text/javascrip